<a href="https://colab.research.google.com/github/ParthibMudi/HAR/blob/main/HAR_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from time import time
import matplotlib as mpl
import matplotlib.pylab as plt
import warnings
warnings.filterwarnings('ignore')


In [9]:
google_drive = '/content/drive/MyDrive/datasets/UCI_HAR/UCI/'



y_train = pd.read_csv(google_drive+'train/y_train.txt', header=None, names=['Activity No.'])
y_test = pd.read_csv(google_drive+'test/y_test.txt', header=None, names=['Activity No.'])

body_acc_x_train = pd.read_csv(google_drive+'train/Inertial Signals/body_acc_x_train.txt', sep='\s+', header=None, index_col=False)
body_acc_y_train = pd.read_csv(google_drive+'train/Inertial Signals/body_acc_y_train.txt', sep='\s+', header=None, index_col=False)
body_acc_z_train = pd.read_csv(google_drive+'train/Inertial Signals/body_acc_z_train.txt', sep='\s+', header=None, index_col=False)
body_gyro_x_train = pd.read_csv(google_drive+'train/Inertial Signals/body_gyro_x_train.txt', sep='\s+', header=None, index_col=False)
body_gyro_y_train = pd.read_csv(google_drive+'train/Inertial Signals/body_gyro_y_train.txt', sep='\s+', header=None, index_col=False)
body_gyro_z_train = pd.read_csv(google_drive+'train/Inertial Signals/body_gyro_z_train.txt', sep='\s+', header=None, index_col=False)
total_acc_x_train = pd.read_csv(google_drive+'train/Inertial Signals/total_acc_x_train.txt', sep='\s+', header=None, index_col=False)
total_acc_y_train = pd.read_csv(google_drive+'train/Inertial Signals/total_acc_y_train.txt', sep='\s+', header=None, index_col=False)
total_acc_z_train = pd.read_csv(google_drive+'train/Inertial Signals/total_acc_z_train.txt', sep='\s+', header=None, index_col=False)

body_acc_x_test = pd.read_csv(google_drive+'test/Inertial Signals/body_acc_x_test.txt', sep='\s+', header=None, index_col=False)
body_acc_y_test = pd.read_csv(google_drive+'test/Inertial Signals/body_acc_y_test.txt', sep='\s+', header=None, index_col=False)
body_acc_z_test = pd.read_csv(google_drive+'test/Inertial Signals/body_acc_z_test.txt', sep='\s+', header=None, index_col=False)
body_gyro_x_test = pd.read_csv(google_drive+'test/Inertial Signals/body_gyro_x_test.txt', sep='\s+', header=None, index_col=False)
body_gyro_y_test = pd.read_csv(google_drive+'test/Inertial Signals/body_gyro_y_test.txt', sep='\s+', header=None, index_col=False)
body_gyro_z_test = pd.read_csv(google_drive+'test/Inertial Signals/body_gyro_z_test.txt', sep='\s+', header=None, index_col=False)
total_acc_x_test = pd.read_csv(google_drive+'test/Inertial Signals/total_acc_x_test.txt', sep='\s+', header=None, index_col=False)
total_acc_y_test = pd.read_csv(google_drive+'test/Inertial Signals/total_acc_y_test.txt', sep='\s+', header=None, index_col=False)
total_acc_z_test = pd.read_csv(google_drive+'test/Inertial Signals/total_acc_z_test.txt', sep='\s+', header=None, index_col=False)

In [11]:
import tensorflow as tf
import numpy as np

# Stacking the 9 features for each time sequence observation (training)
sequence_train = np.dstack((body_acc_x_train, body_acc_y_train, body_acc_z_train,
                            body_gyro_x_train, body_gyro_y_train, body_gyro_z_train,
                            total_acc_x_train, total_acc_y_train, total_acc_z_train))

# Stacking the 9 features for each time sequence observation (testing)
sequence_test = np.dstack((body_acc_x_test, body_acc_y_test, body_acc_z_test,
                           body_gyro_x_test, body_gyro_y_test, body_gyro_z_test,
                           total_acc_x_test, total_acc_y_test, total_acc_z_test))

# Convert to TensorFlow tensors
sequence_train_tensor = tf.convert_to_tensor(sequence_train, dtype=tf.float32)
sequence_test_tensor = tf.convert_to_tensor(sequence_test, dtype=tf.float32)

# Convert labels to tensors (assuming y_train and y_test are pandas Series)
y_train_tensor = tf.convert_to_tensor(y_train.values - 1, dtype=tf.int64)  # Subtracting 1 to zero-index the labels
y_test_true = y_test.values.flatten()  # Flatten if necessary


In [12]:
import tensorflow as tf
import numpy as np

# Define the LSTM model
class LSTMModel(tf.keras.Model):
    def __init__(self):
        super(LSTMModel, self).__init__()
        # LSTM layer
        self.lstm = tf.keras.layers.LSTM(128, return_sequences=False, input_shape=(None, 561))
        # Fully connected layer with 6 output units (for 6 classes) and softmax activation
        self.fc = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, x):
        # Pass input through LSTM layer
        lstm_out = self.lstm(x)
        # Pass the output of LSTM to the fully connected layer
        out = self.fc(lstm_out)
        return out


In [14]:

# Initialize the model
model = LSTMModel()

# Compile the model with Adam optimizer and SparseCategoricalCrossentropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model (assuming batch_size=32 and epochs=10)
history = model.fit(sequence_train_tensor, y_train_tensor, epochs=10, batch_size=32)


Epoch 1/10
230/230 [==============================] - 21s 82ms/step - loss: 1.0689 - accuracy: 0.5524
Epoch 2/10
230/230 [==============================] - 18s 77ms/step - loss: 0.7168 - accuracy: 0.7217
Epoch 3/10
230/230 [==============================] - 17s 75ms/step - loss: 0.4892 - accuracy: 0.8199
Epoch 4/10
230/230 [==============================] - 17s 76ms/step - loss: 0.3574 - accuracy: 0.8698
Epoch 5/10
230/230 [==============================] - 17s 74ms/step - loss: 0.2855 - accuracy: 0.9002
Epoch 6/10
230/230 [==============================] - 17s 74ms/step - loss: 0.2311 - accuracy: 0.9161
Epoch 7/10
230/230 [==============================] - 18s 78ms/step - loss: 0.1989 - accuracy: 0.9267
Epoch 8/10
230/230 [==============================] - 17s 72ms/step - loss: 0.1600 - accuracy: 0.9378
Epoch 9/10
230/230 [==============================] - 18s 80ms/step - loss: 0.1629 - accuracy: 0.9353
Epoch 10/10
230/230 [==============================] - 18s 77ms/step - loss: 0.145

In [15]:

# Evaluate on test data (assuming sequence_test_tensor and y_test_true are available)
y_test_predict = model.predict(sequence_test_tensor)
y_test_predict = np.argmax(y_test_predict, axis=1) + 1  # Convert to class labels


93/93 [==============================] - 3s 24ms/step


In [17]:

# Calculate accuracy
accuracy = np.sum(y_test_predict == y_test_true)*100 / len(y_test_true)
print('Test Accuracy = %.3f' % accuracy)


Test Accuracy = 89.752


In [21]:
import numpy as np


single_sample = sequence_test_tensor[0:1]
prediction = model.predict(single_sample)
predicted_class = np.argmax(prediction, axis=1)
predicted_label = predicted_class[0] + 1
print(f'Predicted class no. : {predicted_label}')
activity = get_activity(predicted_label)
print(f"Predicted class:: {activity}")


1/1 [==============================] - 0s 29ms/step
Predicted class no. : 5
Predicted class:: STANDING


In [19]:
def get_activity(activity_number):
    activity_mapping = {
        1: "WALKING",
        2: "WALKING_UPSTAIRS",
        3: "WALKING_DOWNSTAIRS",
        4: "SITTING",
        5: "STANDING",
        6: "LAYING"
    }

    return activity_mapping.get(activity_number, "Invalid activity number")

